In [2]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

In [3]:
openai.api_key = ""

In [4]:
training_file_name = 'data/NCA/NCA_train_small.jsonl'
validation_file_name = 'data/NCA/NCA_val_small.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-j9eqMKkvQIrT2syqYc2M6pnA
Validation file ID: file-qkwSPAM7CDztvcelN9MjPMlT


In [9]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo-1106",
    suffix="summary-NCA",
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

Job ID: ftjob-GWEpqLhFR2OcD6hPxMyW0YBi
Status: validating_files


In [11]:
#job_id = "ftjob-zZng3nZGavqklcdynf4yS684"
job_id = "ftjob-oisRREmQNrwLt4aU9hEgcaRD"
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])

Job ID: ftjob-oisRREmQNrwLt4aU9hEgcaRD
Status: succeeded
Trained Tokens: 240651


In [12]:

response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-oisRREmQNrwLt4aU9hEgcaRD
Validating training file: file-j9eqMKkvQIrT2syqYc2M6pnA and validation file: file-qkwSPAM7CDztvcelN9MjPMlT
Files validated, moving job to queued state
Fine-tuning job started
Step 1/240: training loss=2.15, validation loss=1.89
Step 11/240: training loss=1.88, validation loss=1.88
Step 21/240: training loss=1.84, validation loss=1.09
Step 31/240: training loss=1.17, validation loss=1.51
Step 41/240: training loss=1.32, validation loss=1.38
Step 51/240: training loss=1.45, validation loss=1.17
Step 61/240: training loss=1.52, validation loss=1.08
Step 71/240: training loss=1.31, validation loss=1.59
Step 81/240: training loss=1.44, validation loss=1.35
Step 91/240: training loss=1.06, validation loss=1.16
Step 101/240: training loss=1.18, validation loss=1.30
Step 111/240: training loss=1.14, validation loss=1.10
Step 121/240: training loss=0.99, validation loss=0.83
Step 131/240: training loss=0.84, validation loss=1.42
Step 141/2

In [13]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-1106:personal:summary-nca:8ebGFgvJ


In [14]:
job_details = openai.FineTuningJob.retrieve(job_id)
print(job_details)

{
  "object": "fine_tuning.job",
  "id": "ftjob-oisRREmQNrwLt4aU9hEgcaRD",
  "model": "gpt-3.5-turbo-1106",
  "created_at": 1704685828,
  "finished_at": 1704687042,
  "fine_tuned_model": "ft:gpt-3.5-turbo-1106:personal:summary-nca:8ebGFgvJ",
  "organization_id": "org-XtFwTtnJF4LiQD00NZQrawyJ",
  "result_files": [
    "file-DC3eRtzIqBToxMzyTsM7A18q"
  ],
  "status": "succeeded",
  "validation_file": "file-qkwSPAM7CDztvcelN9MjPMlT",
  "training_file": "file-j9eqMKkvQIrT2syqYc2M6pnA",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 240651,
  "error": null
}


In [28]:
message_path = "data/NCA/NCA_test_small_message.jsonl"
result_path = "data/NCA/NCA_test_small_resultT3.jsonl"
gt_path = "data/NCA/NCA_test_small_gt.jsonl"

In [29]:
count = 0
with open(message_path, 'r') as message_file:
    with open(result_path, 'w', encoding='utf-8') as result_file:
        for line in message_file:
            print(line)
            count += 1
            json_object = json.loads(line)
            
            response = openai.ChatCompletion.create(model=fine_tuned_model_id, messages=json_object, temperature=0.3, max_tokens=4000)
            print(response)
            result_text = response["choices"][0]["message"]["content"]
            result_text = json.dumps(result_text.strip(), ensure_ascii=False)
            result_file.write(result_text + '\n')

[{"role": "system", "content": "You are a highly skilled doctor with the ability to perfectly summarize a patient's health information. Note: '<PAR>' indicates a new line or paragraph separation. The requirements of the summary as below: 1. **Content**: Your summary should exclusively contain the patient's gender, age, race, current diagnoses, medications, and lab test results. 2. **Format**: Craft the information into a cohesive and informative paragraph. 3. **Accuracy**: Ensure that no required items are omitted. Do not include any past medical history or surgical information; focus solely on the present visit. 4. **Data Availability**: If specific information for the requested items does not exist, clearly indicate this by stating '<item name> data not found' without quotation marks. 5. **Relevance**: Avoid any mention of hospitalization-related matters or details outside the current visit's scope. Your goal is to provide a clear, concise, and accurate summary of the patient's curre

In [30]:
import os

# 原始文件路径
result_file_path = 'D:\\Code\\Jupter\\data\\NCA\\NCA_test_small_resultT3.jsonl'
gt_file_path = 'D:\\Code\\Jupter\\data\\NCA\\NCA_test_small_gt.jsonl'

# 创建保存文本的目录
system_dir = 'D:\\Code\\Jupter\\data\\NCA\\systemT3'
model_dir = 'D:\\Code\\Jupter\\data\\NCA\\modelT3'
os.makedirs(system_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# 将文本分别保存到不同的文件
with open(result_file_path, 'r', encoding='utf-8') as result_file, open(gt_file_path, 'r', encoding='utf-8') as gt_file:
    for i, (result_line, gt_line) in enumerate(zip(result_file, gt_file)):
        with open(os.path.join(system_dir, f"{i}_system.txt"), 'w', encoding='utf-8') as f:
            f.write(result_line.strip())
        with open(os.path.join(model_dir, f"{i}_model.txt"), 'w', encoding='utf-8') as f:
            f.write(gt_line.strip())

In [32]:
result_path = "data/NCA/NCA_test_small_result.jsonl"
gt_path = "data/NCA/NCA_test_small_gt.jsonl"
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from rouge_score import rouge_scorer
import numpy as np
result_list = []
gt_list = []
rouge1 = []
rouge2 = []
rougeL = []

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
with open(result_path, 'r', encoding='utf-8') as result_file, open(gt_path, 'r', encoding='utf-8') as gt_file:
    for result_line, gt_line in zip(result_file, gt_file):
        result_list.append(result_line.strip())
        gt_list.append([gt_line.strip()])
        scores = scorer.score(result_line.strip(), gt_line.strip())
        rouge1.append(scores['rouge1'][2])
        rouge2.append(scores['rouge2'][2])
        rougeL.append(scores['rougeL'][2])
        print(scores)
        
# 计算 BLEU 分数
bleu_score = corpus_bleu(result_list, gt_list)

# 使用 Rouge
avg_rouge1 = np.mean(rouge1)
avg_rouge2 = np.mean(rouge2)
avg_rougeL = np.mean(rougeL)

print(f"Average ROUGE-1 F1: {avg_rouge1:.2f}")
print(f"Average ROUGE-2 F1: {avg_rouge2:.2f}")
print(f"Average ROUGE-L F1: {avg_rougeL:.2f}")


print(f"BLEU Score: {bleu_score}")


{'rouge1': Score(precision=0.4125874125874126, recall=0.5619047619047619, fmeasure=0.4758064516129032), 'rouge2': Score(precision=0.176056338028169, recall=0.2403846153846154, fmeasure=0.2032520325203252), 'rougeL': Score(precision=0.3006993006993007, recall=0.4095238095238095, fmeasure=0.34677419354838707)}
{'rouge1': Score(precision=0.5630252100840336, recall=0.536, fmeasure=0.5491803278688525), 'rouge2': Score(precision=0.2457627118644068, recall=0.23387096774193547, fmeasure=0.2396694214876033), 'rougeL': Score(precision=0.4117647058823529, recall=0.392, fmeasure=0.40163934426229503)}
{'rouge1': Score(precision=0.26277372262773724, recall=0.4090909090909091, fmeasure=0.32000000000000006), 'rouge2': Score(precision=0.11029411764705882, recall=0.1724137931034483, fmeasure=0.13452914798206278), 'rougeL': Score(precision=0.2116788321167883, recall=0.32954545454545453, fmeasure=0.2577777777777777)}
{'rouge1': Score(precision=0.3333333333333333, recall=0.6428571428571429, fmeasure=0.4390

In [35]:
import re

def preprocess(text):
    """文本预处理函数，去除标点并转换为小写"""
    return re.sub(r'[^\w\s]', '', text.lower())

def calculate_coverage(reference_summary, generated_summary):
    """计算 Coverage"""
    key_info_points = reference_summary.split()  # 示例中将每个单词视为一个关键信息点
    processed_generated = preprocess(generated_summary)
    coverage = sum([1 for point in key_info_points if preprocess(point) in processed_generated]) / len(key_info_points)
    return coverage
coverage = []
with open(result_path, 'r', encoding='utf-8') as result_file, open(gt_path, 'r', encoding='utf-8') as gt_file:
    for result_line, gt_line in zip(result_file, gt_file):
        coverage_score = calculate_coverage(gt_line.strip(), result_line.strip())
        print(coverage_score)
        coverage.append(coverage_score)
avg_coverage = np.mean(coverage)

print(f"Average Coverage: {avg_coverage:.2f}")

0.5259259259259259
0.6403508771929824
0.3228346456692913
0.4673913043478261
0.5564516129032258
0.6119402985074627
0.5289256198347108
0.5443037974683544
0.573170731707317
0.48863636363636365
Average Coverage: 0.53
